In [1]:
import sqlite3
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [2]:
# connect database
database = sqlite3.connect("database.sqlite")

In [3]:
# Myleague is Germany 1. Bundesliga (id:7809) 
df = pd.read_sql_query("SELECT date, home_team_api_id,away_team_api_id,home_team_goal,away_team_goal FROM Match WHERE league_id is 7809", database)

In [4]:
#first five data row
df.head()

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,2008-08-15 00:00:00,9823,9790,2,2
1,2008-08-16 00:00:00,8178,9789,2,3
2,2008-08-16 00:00:00,10189,9904,3,0
3,2008-08-16 00:00:00,8721,8722,2,1
4,2008-08-17 00:00:00,9810,8177,0,2


In [5]:
df.shape

(2448, 5)

In [6]:
# looking missing data
df.isnull().sum()

date                0
home_team_api_id    0
away_team_api_id    0
home_team_goal      0
away_team_goal      0
dtype: int64

In [7]:
#statistics
print("Maximum goals scored by the home team: {}".format(df["home_team_goal"].max()))
print("Maximum goals scored by the away team: {}".format(df["away_team_goal"].max()))
print("Average of home team goal: {}".format(df["home_team_goal"].mean()))
print("Average of away team goal: {}".format(df["away_team_goal"].mean()))

Maximum goals scored by the home team: 9
Maximum goals scored by the away team: 8
Average of home team goal: 1.6266339869281046
Average of away team goal: 1.2749183006535947


In [8]:
def result_of_the_match(row):
    if    row["home_team_goal"] < row["away_team_goal"]:
        return "L"
    elif  row["home_team_goal"] > row["away_team_goal"]:
        return "W"
    else:
        return "D"

In [9]:
df['match_result'] = df.apply (lambda row: result_of_the_match(row), axis=1)

In [10]:
df

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,match_result
0,2008-08-15 00:00:00,9823,9790,2,2,D
1,2008-08-16 00:00:00,8178,9789,2,3,L
2,2008-08-16 00:00:00,10189,9904,3,0,W
3,2008-08-16 00:00:00,8721,8722,2,1,W
4,2008-08-17 00:00:00,9810,8177,0,2,L
...,...,...,...,...,...,...
2443,2015-10-17 00:00:00,8406,8262,0,2,L
2444,2015-10-17 00:00:00,8697,9823,0,1,L
2445,2015-10-17 00:00:00,9810,9788,1,5,L
2446,2015-10-18 00:00:00,8722,9904,0,1,L


In [11]:
df.dtypes

date                object
home_team_api_id     int64
away_team_api_id     int64
home_team_goal       int64
away_team_goal       int64
match_result        object
dtype: object

In [12]:
df["result"] = df["match_result"].astype("category").cat.codes

In [13]:
df

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,match_result,result
0,2008-08-15 00:00:00,9823,9790,2,2,D,0
1,2008-08-16 00:00:00,8178,9789,2,3,L,1
2,2008-08-16 00:00:00,10189,9904,3,0,W,2
3,2008-08-16 00:00:00,8721,8722,2,1,W,2
4,2008-08-17 00:00:00,9810,8177,0,2,L,1
...,...,...,...,...,...,...,...
2443,2015-10-17 00:00:00,8406,8262,0,2,L,1
2444,2015-10-17 00:00:00,8697,9823,0,1,L,1
2445,2015-10-17 00:00:00,9810,9788,1,5,L,1
2446,2015-10-18 00:00:00,8722,9904,0,1,L,1


In [14]:
#how many matches did the teams play?
df["home_team_api_id"].value_counts()

9823     136
9790     136
8226     136
9904     136
8178     136
9789     136
8697     136
10269    136
8721     136
10189    136
9788     136
9810     119
9905     119
8177     102
8722     102
8358     102
8165      85
8406      85
9911      34
8350      34
8194      17
8262      17
8460      17
9776      17
8295      17
8357      17
8152      17
9912      17
8398      17
8234      17
Name: home_team_api_id, dtype: int64

In [15]:
#get borussia dortmund data
home = df[df["home_team_api_id"] == 9789]
away = df[df["away_team_api_id"] == 9789]
borussia_dortmund_matches= pd.concat([home, away])
borussia_dortmund_matches

,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,match_result,result
20,2008-11-02 00:00:00,9789,9911,1,1,D,0
37,2008-11-15 00:00:00,9789,9810,4,0,W,2
55,2008-11-30 00:00:00,9789,8721,0,0,D,0
74,2008-12-12 00:00:00,9789,9788,2,1,W,2
82,2009-01-31 00:00:00,9789,8178,1,1,D,0
...,...,...,...,...,...,...,...
2380,2016-05-07 00:00:00,9810,9789,1,0,W,2
2399,2015-09-12 00:00:00,9904,9789,2,4,L,1
2418,2015-09-23 00:00:00,8226,9789,1,1,D,0
2430,2015-10-04 00:00:00,9823,9789,5,1,W,2


In [16]:
n_matches_borussia_dortmund = borussia_dortmund_matches.shape[0]
n_wins = (borussia_dortmund_matches.match_result == "W").sum()
win_rate = (float(n_wins) / n_matches_borussia_dortmund) * 100

In [17]:
print("Total number of matches that Borussia Dortmund played: {}".format(n_matches_borussia_dortmund))
print("Number of matches won by Borussia Dortmund: {}".format(n_wins))
print("Win rate: {}%".format(win_rate))

Total number of matches that Borussia Dortmund played: 272
Number of matches won by Borussia Dortmund: 123
Win rate: 45.220588235294116%


In [18]:
print("The maximum home goals scored by Borussia Dortmund: {}".format(borussia_dortmund_matches["home_team_goal"].max()))
print("The maximum away goals scored by Borussia Dortmund: {}".format(borussia_dortmund_matches["away_team_goal"].max()))
print("The home goal average of Borussia Dortmund scored: {}".format(borussia_dortmund_matches["home_team_goal"].mean()))
print("The away goal average of Borussia Dortmund scored: {}".format(borussia_dortmund_matches["away_team_goal"].mean()))

The maximum home goals scored by Borussia Dortmund: 6
The maximum away goals scored by Borussia Dortmund: 6
The home goal average of Borussia Dortmund scored: 1.7205882352941178
The away goal average of Borussia Dortmund scored: 1.3419117647058822


In [19]:
train, test = train_test_split(borussia_dortmund_matches, test_size=0.2, random_state=25)

In [20]:
print(f"Number of training: {train.shape[0]}")
print(f"Number of testing: {test.shape[0]}")

Number of training: 217
Number of testing: 55


In [21]:
predictors = ["home_team_api_id", "away_team_api_id"]

In [22]:
knear = KNeighborsClassifier(n_neighbors=5)
knear.fit(train[predictors], train["result"])
knear_prediction = knear.predict(test[predictors])
knear_acc = accuracy_score(test["result"], knear_prediction)
knear_score = precision_score(test["result"], knear_prediction, average="micro")

In [23]:
randf = RandomForestClassifier(n_estimators=70, min_samples_split=10, random_state=1)
randf.fit(train[predictors], train["result"])
randf_prediction = randf.predict(test[predictors])
randf_acc = accuracy_score(test["result"], randf_prediction)
randf_score = precision_score(test["result"], randf_prediction, average="micro")

In [24]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(train[predictors], train["result"])
gb_prediction = gb.predict(test[predictors])
gb_acc = accuracy_score(test["result"], gb_prediction)
gb_score = precision_score(test["result"], gb_prediction, average="micro")

In [25]:
print("K-nearest Classifier Accuracy Score: {}%".format(knear_acc*100))
print("Random Forrest Classifier Accuracy Score: {}%".format(randf_acc*100))
print("Gradient Boosting Classifier Accuracy Score: {}%".format(gb_acc*100))

K-nearest Classifier Accuracy Score: 50.90909090909091%
Random Forrest Classifier Accuracy Score: 54.54545454545454%
Gradient Boosting Classifier Accuracy Score: 52.72727272727272%


In [26]:
matrix = pd.DataFrame(dict(actual=test["result"], predicted=knear_prediction))
pd.crosstab(index=matrix["actual"], columns=matrix["predicted"])

predicted,0,1,2
actual,,,
0,1,4,7
1,5,12,5
2,2,4,15


In [27]:
matrix = pd.DataFrame(dict(actual=test["result"], predicted=randf_prediction))
pd.crosstab(index=matrix["actual"], columns=matrix["predicted"])

predicted,0,1,2
actual,,,
0,0,6,6
1,2,14,6
2,2,3,16


In [28]:
matrix = pd.DataFrame(dict(actual=test["result"], predicted=gb_prediction))
pd.crosstab(index=matrix["actual"], columns=matrix["predicted"])

predicted,0,1,2
actual,,,
0,0,3,9
1,3,11,8
2,1,2,18


In [29]:
print("K-nearest Precision score: {}%".format(knear_score*100))
print("Random Forrest Precision score: {}%".format(randf_score*100))
print("Gradient Boosting Precision score: {}%".format(gb_score*100))

K-nearest Precision score: 50.90909090909091%
Random Forrest Precision score: 54.54545454545454%
Gradient Boosting Precision score: 52.72727272727272%
